In [152]:
import numpy as np
import pandas as pd

from importlib import reload

In [153]:
mice = pd.read_csv("../Data/imputed_data.mice.csv")

In [154]:
import warnings
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(df):
    X = df.dropna().select_dtypes(include='number') #will include all the numeric types
    vif = pd.DataFrame()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['Column'] = X.columns
    vif = vif.set_index('Column').dropna()
    vif = vif.sort_values('VIF', ascending=False).T
    return vif.round(2)

In [155]:
df = mice[mice.imp == 1].drop(['imp', 'id', 'full', 'lobe', 'other'], axis=1)
df = df.set_index('name')
df.head()

,tspk1,tsfw1,lspk1,lsfw1,tspk2,tsfw2,lspk2,lsfw2,tspk3,tsfw3,lspk3,lsfw3,teosVolume,teosVolPct,quality
name,,,,,,,,,,,,,,,
trial_02,514,37.365087,794,94.413732,515.0,36.940846,820.0,83.674204,519.0,81.198341,799.0,101.710992,125.0,20,2
trial_03,519,64.129804,779,98.099647,518.0,46.944391,786.0,89.298429,528.0,80.601838,783.0,104.473499,125.0,20,2
trial_04,511,38.079594,813,90.520202,514.0,42.576748,812.0,90.893479,517.0,40.194940,810.0,96.933421,75.0,20,3
trial_05,519,46.014336,772,105.349272,521.0,51.028356,765.0,106.424655,522.0,52.401751,771.0,118.127834,75.0,20,3
trial_06,512,35.237926,832,79.944318,510.0,32.994429,823.0,85.849053,515.0,65.638905,809.0,137.124369,75.0,20,2


In [156]:
from model import features
reload(features)

<module 'model.features' from 'D:\\Dropbox\\work-Proj\\Proj-UV-GNR-ML\\code\\model\\features.py'>

In [157]:
ddf = features.Differences(df)
idf = features.InverseDifferences(ddf)

In [158]:
def find_noncollinear_features(df, x0, columns, vif_cutoff = 8):
    """ Try to add columns one by one to dataframe without creating multicollinearity.
        Return the list of features that passed VIF check.
    """
    candidate = []
    collinear = []
    passed = x0.copy()
    for col in columns:
        if col in passed:
            continue
        try:
            vif = calc_vif(df[passed + [col]]).drop(col, axis=1)
        except:
            continue
        if vif.iloc[0, 0] < vif_cutoff:
            passed.append(col)
        else:
            candidate.append(col)
            collinear.append(vif.columns[0])

    failed = pd.DataFrame({'linearColumn': collinear}, index=candidate)
    return calc_vif(df[passed]), failed

In [159]:
x0 = ['teosVolPct', 'teosVolume']

In [169]:
ddf

,tspk1,tsfw1,lspk1,lsfw1,tspk2,tsfw2,lspk2,lsfw2,tspk3,tsfw3,...,dp33,dp21,dp31,dp32,dw11,dw22,dw33,dw21,dw31,dw32
name,,,,,,,,,,,,,,,,,,,,,
trial_02,514,37.365087,794,94.413732,515.0,36.940846,820.0,83.674204,519.0,81.198341,...,280.0,25.0,0.0,-25.0,57.048645,46.733358,20.512651,-10.315287,-36.535995,-26.220707
trial_03,519,64.129804,779,98.099647,518.0,46.944391,786.0,89.298429,528.0,80.601838,...,255.0,8.0,-5.0,-13.0,33.969844,42.354038,23.871661,8.384194,-10.098183,-18.482377
trial_04,511,38.079594,813,90.520202,514.0,42.576748,812.0,90.893479,517.0,40.194940,...,293.0,-4.0,-9.0,-5.0,52.440608,48.316731,56.738481,-4.123877,4.297873,8.421750
trial_05,519,46.014336,772,105.349272,521.0,51.028356,765.0,106.424655,522.0,52.401751,...,249.0,-9.0,-4.0,5.0,59.334936,55.396300,65.726083,-3.938636,6.391148,10.329784
trial_06,512,35.237926,832,79.944318,510.0,32.994429,823.0,85.849053,515.0,65.638905,...,294.0,-7.0,-26.0,-19.0,44.706392,52.854624,71.485464,8.148232,26.779072,18.630840
trial_07,517,42.651335,797,91.664429,515.0,42.286900,798.0,91.998087,518.0,45.651200,...,273.0,3.0,-7.0,-10.0,49.013094,49.711187,52.245409,0.698093,3.232316,2.534223
trial_08,517,48.076381,717,88.568606,512.0,37.261999,720.0,92.623238,518.0,40.881576,...,199.0,8.0,-1.0,-9.0,40.492225,55.361240,55.403177,14.869015,14.910953,0.041938
trial_09,522,42.957700,772,92.603371,514.0,39.511930,786.0,94.785207,524.0,49.106953,...,250.0,22.0,0.0,-22.0,49.645671,55.273277,55.366546,5.627607,5.720875,0.093268
trial_14,515,39.551512,737,93.464411,516.0,42.769507,745.0,97.551287,525.0,57.896645,...,249.0,7.0,27.0,20.0,53.912899,54.781780,79.227723,0.868881,25.314824,24.445943


In [168]:
find_noncollinear_features(ddf, x0, ddf.columns)

(Column  teosVolPct  lspk1  teosVolume  lp31  dw31
 VIF           7.98   7.84        2.79  1.55  1.39,
         linearColumn
 tspk1     teosVolPct
 tsfw1     teosVolPct
 lsfw1          lspk1
 tspk2          lspk1
 tsfw2          lspk1
 lspk2          lspk1
 lsfw2          lspk1
 tspk3          lspk1
 tsfw3          lspk1
 lspk3          lspk1
 lsfw3          lspk1
 quality        lspk1
 lp21           lspk1
 lp32           lspk1
 tp21           lspk1
 tp31      teosVolPct
 tp32      teosVolPct
 lw21           lspk1
 lw31      teosVolPct
 lw32           lspk1
 tw21           lspk1
 tw31      teosVolPct
 tw32      teosVolPct
 dp11           lspk1
 dp22           lspk1
 dp33           lspk1
 dp21           lspk1
 dp31            lp31
 dp32           lspk1
 dw11           lspk1
 dw22           lspk1
 dw33           lspk1
 dw21           lspk1
 dw32           lspk1)

In [161]:
find_noncollinear_features(idf, x0, ddf.columns)[0]

Column,teosVolPct,lspk1,teosVolume,lp31,dw31
VIF,7.98,7.84,2.79,1.55,1.39


In [162]:
find_noncollinear_features(idf, x0, idf.columns)[0]

Column,teosVolPct,lspk1,teosVolume,lp31,idw33,dw31
VIF,7.98,7.9,2.85,2.15,1.9,1.49


In [163]:
from sklearn.preprocessing import StandardScaler

In [164]:
sclr = StandardScaler().fit(idf)
sdf = sclr.transform(idf)
sdf = pd.DataFrame(sdf, index=idf.index, columns=idf.columns)

In [165]:
find_noncollinear_features(sdf, x0, idf.columns)[0]

Column,lspk2,lspk1,tspk2,tsfw2,tsfw1,lsfw1,tspk1,teosVolume,teosVolPct
VIF,8.02,7.46,5.21,4.66,3.32,3.13,2.12,1.76,1.42
